# Линейная регрессия

Здесь хороший повод поговорить о математике, но меня просили не. Что ж, надеюсь, это не помешает вам сопоставить теорию с практикой. 

Суть: у нас есть несколько параметров и есть некоторое числовое значение, зависимость которого от этих самых параметров мы и хотим узнать. Например: есть количество квартир в доме, есть площадь парковки возле дома и есть площадь конкретной квартиры. По этим трём параметрам нужно найти стоимость этой квартиры. Пара из трёх параметров и цены квартиры составляют одну запись/один индивид. Таких индивидов у нас несколько (желательно довольно много). 

## Цели линейной регрессии

Собственно линейную регрессию используют для двух целей:
* Предсказать значение нового индивида при известных параметрах
* Объяснить как влияет каждый из параметров на искомое значение

## Когда её используют

Обычно её используют для получения первых выводов из данных, перед тем как начать использовать ~~нормальные~~ более сложные модели. 

## Какой у нас план?

В виду ограничений, будем пытаться разобраться по ходу, надеюсь, вам будет понятно. Вообще будет здорово, если освежите теоретические основы, но постараюсь всё показать на живом примере.

## Библиотека 

По традиции, нам нужно для начала установить библиотеку. В нашем случае на выбор предоставляется две библиотеки: `statsmodels` и `scikit-learn` (читают *склёрн*). Первая хороша для тех, кто дружит с матстатом, поэтому это не наш случай и выбираем вторую. Функционал этих библиотек неограничен исключительно линейной регрессией, но пока мы рассмотрим только её. 

In [ ]:
# !pip install scikit-learn

### Импорт

Внезапно, импортируем мы её не как `import scikit-learn` и даже не `import sklearn`. Библиотека довольно объёмна, а нам нужны только отдельные её модули, даже отдельные части её модулей. Для большей целостности импортируем всё необходимое здесь, а по ходу дела буду объяснять, что именно мы здесь набрали.

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Начало работы / обработка датасета

Самое первое, что необходимо сделать так это узнать какие данные у нас есть и что мы хотим вообще найти.

In [6]:
# загружаем датасет из библиотеки sci-kit learn 
# эту функцию мы импортировали из модуля datasets 
data = fetch_california_housing() 
data


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 